In [ ]:
import pandas as pd
# import modin.pandas as pd
from pathlib import Path
import os
import timeit

In [ ]:
import sys
sys.path.insert(0, '/src')
from shared.load_raw_data import get_genric_file_names
from shared.constants import CLIENT
from eliot import to_file
to_file(sys.stdout)

In [ ]:
# Strip client name and get the actual dataframe names

data_path = Path('/data/raw')
client_file_types = get_genric_file_names(data_path, CLIENT)

print(client_file_types)

In [ ]:
"""
- When using other clients data, merge respective files across all 
  given clients and rename them removing the client name
- Append client_name to masterpatientid
- Add a client column
"""
start_time = timeit.default_timer()
for ft in client_file_types:
    try:
        # Fetch same file across client
        client_files = data_path.glob(f'*_{ft}.parquet')

        df = pd.DataFrame()
        
        # Loop through all the files and combine them
        for f in client_files:
            client = f.name.split('_')[0]
            client_df = pd.read_parquet(f)
            client_df['masterpatientid'] = client_df['masterpatientid'].apply(lambda x: client + '_' + str(x))
            client_df['client'] = client
            df = df.append(client_df, ignore_index=True)
            print(f, len(client_df), len(df))

        if ft == 'patient_demographics':
            df['dateofbirth'] = df['dateofbirth'].astype('datetime64[ms]')
            
        if ft == 'patient_diagnosis':
            df['onsetdate'] = df['onsetdate'].astype('datetime64[ms]')
            
        df.to_parquet(data_path/f'{ft}.parquet')
        print('============================')
    except Exception as e:
        print(ft, 'failed:', e)
print(f"{timeit.default_timer() - start_time} seconds")

### ======================== TESTING ================================

In [ ]:
# Load generic named Training data which is cached in local folders
# from shared.load_raw_data import fetch_training_cache_data

# result_dict = fetch_training_cache_data(client=CLIENT, generic=True)
# for key, value in result_dict.items():
#     print(f'{key} : {result_dict[key].info()}')

In [ ]:
# Remove all newly generated parquet files

# for ft in client_file_types:
#     os.remove(data_path/f'{ft}.parquet')